In [14]:
# !source ../setup.sh

In [2]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
from utils import DataWrapper, Visualizer
from src.prepare_dataset import prepare_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# from src.models.baseline.tree_models import XGBoost, LightGBM
# from src.models.baseline.baseline_models import LinearModel, KNN, SVM, DecisionTree, RandomForest
sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings("ignore")

Matplotlib is building the font cache; this may take a moment.


In [3]:
prepare_data("../data/final_dataset.csv", "./tmp")

../data/final_dataset.csv


2024-07-26 08:46:15.983 | INFO     | src.prepare_dataset:prepare_data:31 - Dropped the unnecessary columns
2024-07-26 08:46:15.984 | INFO     | src.prepare_dataset:prepare_data:36 - Dataset size BEFORE dropping the rows by date: 372084
2024-07-26 08:46:16.619 | INFO     | src.prepare_dataset:prepare_data:38 - Dataset size AFTER dropping the rows by date: 369697
2024-07-26 08:46:16.823 | INFO     | src.prepare_dataset:prepare_data:56 - Overall encoded 5 cateforical features
2024-07-26 08:46:16.823 | INFO     | src.prepare_dataset:prepare_data:57 - ['school_hours', 'school_day', 'break_time', 'classroom_type', 'power_on']
2024-07-26 08:46:17.360 | INFO     | src.prepare_dataset:prepare_data:79 - Filled the missing values of variables
2024-07-26 08:46:17.371 | INFO     | src.prepare_dataset:prepare_data:81 - Sliced the data into train, valid, test splits with the proportion: 
2024-07-26 08:46:17.422 | INFO     | src.prepare_dataset:prepare_data:85 - Applied the MinMaxScaler to the dataset

In [4]:
train = pd.read_csv("./tmp/train.csv")
val = pd.read_csv("./tmp/valid.csv")
test = pd.read_csv("./tmp/test.csv")
keep_tmstamp = False
if keep_tmstamp:
    train['tmstamp'] = train['tmstamp'].astype('category')
    val['tmstamp'] = val['tmstamp'].astype('category')
    test['tmstamp'] = test['tmstamp'].astype('category')
else:
    train = train.drop(columns=['tmstamp'])
    val = val.drop(columns=['tmstamp'])
    test = test.drop(columns=['tmstamp'])

In [5]:
X_train, y_train = train.loc[:, train.columns != 'measured_t'], train['measured_t']
X_val, y_val = val.loc[:, val.columns != 'measured_t'], val['measured_t']
X_test, y_test = test.loc[:, test.columns != 'measured_t'], test['measured_t']

In [6]:
X_train

,classroom_type,school_day,school_hours,break_time,power_on,trackerwm_avg,shadowwm_avg,sunwm_avg,shadowbandwm_avg,dnicalc_avg,...,ws_ms_s_wvt,winddir_d1_wvt,winddir_sd1_wvt,bp_mb_avg,uva_avg,uvb_avg,Month,Day,Hour,Minute
0,0,1,0,0,1,0.000000,0.764375,0.010247,0.061300,0.282270,...,0.595232,0.000000,0.748538,0.106834,0.008796,0.074619,3,16,7,17
1,1,1,0,0,1,0.000000,0.764626,0.010425,0.061711,0.282342,...,0.595232,0.000000,0.748538,0.106678,0.008796,0.074433,3,16,7,18
2,1,1,0,0,1,0.000000,0.764751,0.010425,0.061754,0.282256,...,0.595232,0.000000,0.748538,0.105712,0.008642,0.074248,3,16,7,19
3,0,1,0,0,1,0.000730,0.766587,0.011724,0.064445,0.282355,...,0.621405,0.005905,0.764025,0.107146,0.008488,0.074248,3,16,7,27
4,1,1,0,0,1,0.000332,0.774045,0.016519,0.079077,0.282305,...,0.595232,0.000000,0.748538,0.106023,0.008951,0.074248,3,16,7,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258782,0,0,0,0,1,0.016931,0.895718,0.096094,0.321443,0.282035,...,0.700235,0.059070,0.896844,0.479983,0.611368,0.799229,9,16,13,24
258783,5,0,0,0,1,0.120073,0.926651,0.161734,0.372700,0.288066,...,0.757264,0.064367,0.825476,0.481066,0.611381,0.799245,9,16,13,26
258784,2,0,0,0,1,0.120073,0.926651,0.161734,0.372700,0.288066,...,0.757264,0.064367,0.825476,0.481066,0.611381,0.799245,9,16,13,26
258785,1,0,0,0,1,0.006344,0.903829,0.103442,0.341516,0.282699,...,0.767009,0.061584,0.817416,0.481311,0.607714,0.799289,9,16,13,28


In [7]:
# Create regression matrices
cat = False
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=cat)
dval_reg = xgb.DMatrix(X_val, y_val, enable_categorical=cat)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=cat)

In [9]:
# Define hyperparameters
# GPU: "tree_method": "gpu_hist"
# No GPU: "tree_method": "hist"
params = {"objective": "reg:squarederror", "tree_method": "hist"}
evals = [(dtrain_reg, "train"), (dval_reg, "validation")]

n = 10000
model_xgb = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
    evals=evals,
    verbose_eval=10,
    early_stopping_rounds=50
)

[0]	train-rmse:0.09044	validation-rmse:0.10197
[10]	train-rmse:0.04663	validation-rmse:0.06277
[20]	train-rmse:0.04309	validation-rmse:0.06280
[30]	train-rmse:0.04174	validation-rmse:0.06300
[40]	train-rmse:0.04084	validation-rmse:0.06508
[50]	train-rmse:0.04017	validation-rmse:0.06420
[60]	train-rmse:0.03950	validation-rmse:0.06411
[65]	train-rmse:0.03926	validation-rmse:0.06377


In [10]:
pred_xgb = model_xgb.predict(dtest_reg)
rmse_xgb = mean_squared_error(y_test, pred_xgb, squared=False)
r2_xgb = r2_score(y_test, pred_xgb)

print(f"RMSE of the XGBoost model: {rmse_xgb:.5f}")
print(f"R2 of the XGBoost model: {r2_xgb:.5f}")

RMSE of the XGBoost model: 0.14392
R2 of the XGBoost model: -0.68412


In [11]:
# Fitting Random Forest Regression to the dataset
model_rfr = RandomForestRegressor(n_estimators=100, random_state=0, oob_score=True)

# Fit the regressor with x and y data
model_rfr.fit(X_train, y_train)

RandomForestRegressor(oob_score=True, random_state=0)

In [12]:
pred_rfr = model_rfr.predict(X_test)
rmse_rfr = mean_squared_error(y_test, pred_rfr, squared=False)
r2_rfr = r2_score(y_test, pred_rfr)

print(f"RMSE of the Random Forest Regression model: {rmse_rfr:.5f}")
print(f"R2 of the Random Forest Regression model: {r2_rfr:.5f}")

RMSE of the Random Forest Regression model: 0.14647
R2 of the Random Forest Regression model: -0.74426
